# BATTLE OF CITIES

# #Introduction

London and Paris are quite the popular tourist and vacation destinations for people all around the world. They are diverse and multicultural and offer a wide variety of experiences that is widely sought after. We try to group the neighbourhoods of London and Paris respectively and draw insights to what they look like now.

# BUSINESS PROBLEM

The aim is to help tourists choose their destinations depending on the experiences that the neighbourhoods have to offer and what they would want to have. This also helps people make decisions if they are thinking about migrating to London or Paris or even if they want to relocate neighbourhoods within the city. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer.

## London Data

To derive our solution, We scrape our data from https://en.wikipedia.org/wiki/List_of_areas_of_London

This wikipedia page has information about all the neighbourhoods, we limit it London.

borough : Name of Neighbourhood
town : Name of borough
post_code : Postal codes for London.
This wikipedia page lacks information about the geographical locations. To solve this problem we use ArcGIS API

## Paris Data

To derive our solution, We leverage JSON data available at https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e

The JSON file has data about all the neighbourhoods in France, we limit it to Paris.

postal_code : Postal codes for France
nom_comm : Name of Neighbourhoods in France
nom_dept : Name of the boroughs, equivalent to towns in France
geo_point_2d : Tuple containing the latitude and longitude of the Neighbourhoods.


In [3]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means for the clustering stage
from sklearn.cluster import KMeans

In [4]:
pip install folium

     |████████████████████████████████| 94 kB 6.5 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import folium

Expolore LONDON DATA

In [6]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

In [7]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                       Download coordinates as: KML,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 526         Woolwich                          Greenwich          LONDON   
 527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 529          Yeading                         Hillingdon           HAYES   
 

In [8]:
wiki_london_data = wiki_london_data[1]
wiki_london_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


DATA PROCESSING

In [9]:

wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_london_data

,Location,London borough,Post_town,Postcode district,Dial code,OS_grid_ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [10]:
df1 = wiki_london_data.drop( [ wiki_london_data.columns[0], wiki_london_data.columns[4], wiki_london_data.columns[5] ], axis=1)

In [11]:
df1.head()

,London borough,Post_town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [12]:
df1.columns = ['borough','town','post_code']
df1

,borough,town,post_code
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
526,Greenwich,LONDON,SE18
527,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
528,Hammersmith and Fulham,LONDON,W12
529,Hillingdon,HAYES,UB4


In [13]:
df1['borough'] = df1['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
526,Greenwich,LONDON,SE18
527,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
528,Hammersmith and Fulham,LONDON,W12
529,Hillingdon,HAYES,UB4


In [14]:
df1.shape

(531, 3)

In [15]:
df1 = df1[df1['town'].str.contains('LONDON')]
df1

,borough,town,post_code
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20
...,...,...,...
521,Redbridge,LONDON,"IG8, E18"
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8
525,Barnet,LONDON,N12
526,Greenwich,LONDON,SE18


In [16]:
pip install arcgis

Note: you may need to restart the kernel to use updated packages.


In [17]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [18]:
def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England, GBR'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [19]:
c = get_x_y_uk('SE2')

In [20]:
c

'51.492450000000076,0.12127000000003818'

In [21]:
geo_coordinates_uk = df1['post_code']    
geo_coordinates_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
521    IG8, E18
522         IG8
525         N12
526        SE18
528         W12
Name: post_code, Length: 308, dtype: object

In [22]:
coordinates_latlng_uk = geo_coordinates_uk.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_uk

0       51.492450000000076,0.12127000000003818
1        51.51324000000005,-0.2674599999999714
6       51.51200000000006,-0.08057999999994081
7       51.51651000000004,-0.11967999999995982
9       51.41009000000008,-0.05682999999993399
                        ...                   
521    51.589770000000044,0.030520000000024083
522      51.50642000000005,-0.1272099999999341
525     51.615920000000074,-0.1767399999999384
526      51.48207000000008,0.07143000000002075
528      51.50645000000003,-0.2369099999999662
Name: post_code, Length: 308, dtype: object

In [23]:
lat_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[0])
lat_uk

0      51.492450000000076
1       51.51324000000005
6       51.51200000000006
7       51.51651000000004
9       51.41009000000008
              ...        
521    51.589770000000044
522     51.50642000000005
525    51.615920000000074
526     51.48207000000008
528     51.50645000000003
Name: post_code, Length: 308, dtype: object

In [24]:
lng_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[1])
lng_uk

0       0.12127000000003818
1       -0.2674599999999714
6      -0.08057999999994081
7      -0.11967999999995982
9      -0.05682999999993399
               ...         
521    0.030520000000024083
522     -0.1272099999999341
525     -0.1767399999999384
526     0.07143000000002075
528     -0.2369099999999662
Name: post_code, Length: 308, dtype: object

In [25]:
london_merged = pd.concat([df1,lat_uk.astype(float), lng_uk.astype(float)], axis=1)
london_merged.columns= ['borough','town','post_code','latitude','longitude']
london_merged

,borough,town,post_code,latitude,longitude
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746
6,City,LONDON,EC3,51.51200,-0.08058
7,Westminster,LONDON,WC2,51.51651,-0.11968
9,Bromley,LONDON,SE20,51.41009,-0.05683
...,...,...,...,...,...
521,Redbridge,LONDON,"IG8, E18",51.58977,0.03052
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.50642,-0.12721
525,Barnet,LONDON,N12,51.61592,-0.17674
526,Greenwich,LONDON,SE18,51.48207,0.07143


## PARIS DATA

In [26]:
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print("Data Downloaded!")

Data Downloaded!


In [27]:
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


In [28]:

paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


In [29]:
df_2 = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df_2

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"
...,...,...,...,...
1295,77520,CESSOY-EN-MONTOIS,SEINE-ET-MARNE,"[48.50730730461658, 3.138844194183689]"
1296,93420,VILLEPINTE,SEINE-SAINT-DENIS,"[48.95902025378707, 2.536306342059409]"
1297,77130,CANNES-ECLUSE,SEINE-ET-MARNE,"[48.36403767307805, 2.990786679832767]"
1298,78930,VILLETTE,YVELINES,"[48.92627887061508, 1.6937417245662671]"


In [30]:

df_paris = df_2[df_2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"
5,75004,PARIS-4E-ARRONDISSEMENT,PARIS,"[48.854228281954754, 2.357361938142205]"
6,75001,PARIS-1ER-ARRONDISSEMENT,PARIS,"[48.8626304851685, 2.336293446550539]"
7,75008,PARIS-8E-ARRONDISSEMENT,PARIS,"[48.87252726662346, 2.312582560420059]"
8,75013,PARIS-13E-ARRONDISSEMENT,PARIS,"[48.82871768452136, 2.362468228516128]"
9,75012,PARIS-12E-ARRONDISSEMENT,PARIS,"[48.83515623066034, 2.419807034965275]"


In [31]:

df_paris['geo_point_2d'][0]

[48.87689616237872, 2.337460241388529]

In [32]:
temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.86305413181178, 2.359361058970589]
4      [48.84896809191946, 2.332670898588416]
5     [48.854228281954754, 2.357361938142205]
6       [48.8626304851685, 2.336293446550539]
7      [48.87252726662346, 2.312582560420059]
8      [48.82871768452136, 2.362468228516128]
9      [48.83515623066034, 2.419807034965275]
10    [48.844508659617546, 2.349859385560182]
11     [48.88686862295828, 2.384694327870042]
12     [48.86318677744551, 2.400819826729021]
13     [48.87602855694339, 2.361112904561707]
14     [48.86039876035177, 2.262099559395783]
15    [48.892735074561706, 2.348711933867703]
16     [48.88733716648682, 2.307485559493426]
17     [48.85608259819694, 2.312438687733857]
18     [48.84015541860987, 2.293559372435076]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [33]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

In [34]:
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5     48.854228281954754
6       48.8626304851685
7      48.87252726662346
8      48.82871768452136
9      48.83515623066034
10    48.844508659617546
11     48.88686862295828
12     48.86318677744551
13     48.87602855694339
14     48.86039876035177
15    48.892735074561706
16     48.88733716648682
17     48.85608259819694
18     48.84015541860987
19     48.82899321160942
Name: geo_point_2d, dtype: object

In [35]:

paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.357361938142205
6      2.336293446550539
7      2.312582560420059
8      2.362468228516128
9      2.419807034965275
10     2.349859385560182
11     2.384694327870042
12     2.400819826729021
13     2.361112904561707
14     2.262099559395783
15     2.348711933867703
16     2.307485559493426
17     2.312438687733857
18     2.293559372435076
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [36]:
paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

In [37]:
paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data

,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
5,75004,PARIS-4E-ARRONDISSEMENT,PARIS,48.854228,2.357362
6,75001,PARIS-1ER-ARRONDISSEMENT,PARIS,48.862630,2.336293
7,75008,PARIS-8E-ARRONDISSEMENT,PARIS,48.872527,2.312583
8,75013,PARIS-13E-ARRONDISSEMENT,PARIS,48.828718,2.362468
9,75012,PARIS-12E-ARRONDISSEMENT,PARIS,48.835156,2.419807


COORDINATES FOR PARIS

In [38]:

paris = geocode(address='Paris, France, FR')[0]
paris_lng_coords = paris['location']['x']
paris_lat_coords = paris['location']['y']
print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geolocation of Paris:  48.85341000000005 2.3488000000000397


COORSINATES FOR LONDON

In [39]:
london = geocode(address='London, England, GBR')[0]
london_lng_coords = london['location']['x']
london_lat_coords = london['location']['y']
london_lng_coords
london_lat_coords

51.50642000000005

In [40]:
london_lng_coords

-0.1272099999999341

In [41]:
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_London

# adding markers to map
for latitude, longitude, borough, town in zip(london_merged['latitude'], london_merged['longitude'], london_merged['borough'], london_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

In [47]:
CLIENT_ID = 'FBR0NC431IJ15OBT5VXVQU4DGS0QZL2GQ3YGIW5VYNHCPJHN' 
CLIENT_SECRET = '5IZY0ZHWOPK4MUN1HOEUTU14G33LHN0V24Y2CDO1CSAV00IY'
VERSION = '20180605' # Foursquare API version

In [55]:
LIMIT=50

def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
venues_in_London = getNearbyVenues(london_merged['borough'], london_merged['latitude'], london_merged['longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and Chelsea, Hammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames


In [57]:
venues_in_London.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,Supermarket
1,"Bexley, Greenwich",51.49245,0.12127,Bean @ Work,Coffee Shop
2,"Bexley, Greenwich",51.49245,0.12127,The Abbey Arms,Pub
3,"Bexley, Greenwich",51.49245,0.12127,Costcutter,Convenience Store
4,"Ealing, Hammersmith and Fulham",51.51324,-0.26746,london hijabs,Clothing Store


In [58]:
venues_in_London.shape

(3121, 5)

In [59]:
venues_in_London.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
African Restaurant,Hammersmith and Fulham,51.50645,-0.23691,Red Sea Restaurant
American Restaurant,Barnet,51.63261,-0.17562,Hudson American Brasserie
Argentinian Restaurant,Wandsworth,51.42170,-0.20796,Buenos Aires Argentine Steakhouse Wimbledon
Art Gallery,Southwark,51.50642,-0.06779,Mall Galleries
Art Museum,Kensington and Chelsea,51.49807,-0.17404,Victoria and Albert Museum (V&A)
...,...,...,...,...
Warehouse Store,Wandsworth,51.60498,-0.20796,Tiens4u
Wine Bar,Wandsworth,51.55570,-0.02467,WC
Wine Shop,"Lambeth, Wandsworth",51.59107,0.00275,Theatre of Wine


In [60]:
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat

,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Train Station,Tram Station,Turkish Restaurant,Used Bookstore,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,...,Train Station,Tram Station,Turkish Restaurant,Used Bookstore,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Ealing, Hammersmith and Fulham",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:

London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()

,Neighbourhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,...,Train Station,Tram Station,Turkish Restaurant,Used Bookstore,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Barnet,0.0,0.010638,0.0,0.0,0.0,0.0,0.024823,0.0,0.014184,...,0.0,0.0,0.014184,0.0,0.003546,0.0,0.0,0.003546,0.0,0.0
1,"Barnet, Brent, Camden",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,Bexley,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,"Bexley, Greenwich",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,"Bexley, Greenwich",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0


In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [65]:
# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Grocery Store,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Bus Stop,Hotel,Asian Restaurant,Supermarket
1,"Barnet, Brent, Camden",Clothing Store,Bus Station,Hardware Store,Convenience Store,Supermarket,Bakery,Yoga Studio,Fishing Store,Fish Market,Fish & Chips Shop
2,Bexley,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
3,"Bexley, Greenwich",Bus Stop,Yoga Studio,Food Court,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,"Bexley, Greenwich",Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant


# MODEL BUILDING


In [66]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(n_clusters=5, random_state=0)

In [67]:
kmeans_london.labels_

array([1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 3, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1], dtype=int32)

In [68]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

In [69]:
london_data = london_merged

london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')

london_data.head()

,borough,town,post_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127,1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746,4.0,Bed & Breakfast,Hotel,Pub,Clothing Store,Ethiopian Restaurant,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,City,LONDON,EC3,51.51200,-0.08058,2.0,Coffee Shop,Italian Restaurant,Hotel,Gym / Fitness Center,Pub,Fast Food Restaurant,Sandwich Place,Wine Bar,Salad Place,Restaurant
7,Westminster,LONDON,WC2,51.51651,-0.11968,2.0,Coffee Shop,Hotel,Pub,Sandwich Place,Salad Place,Restaurant,Gym / Fitness Center,Indian Restaurant,Portuguese Restaurant,Café
9,Bromley,LONDON,SE20,51.41009,-0.05683,2.0,Hotel,Café,Fish & Chips Shop,Gym / Fitness Center,Supermarket,Portuguese Restaurant,Pub,Bistro,Yoga Studio,Ethiopian Restaurant


In [70]:
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])

In [71]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['latitude'], london_data_nonan['longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london

# EXAMINING CLUSTER


In [72]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
45,"BEXLEYHEATH, LONDON",1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
124,LONDON,1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
291,"LONDON, SIDCUP",1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
505,LONDON,1.0,Convenience Store,Supermarket,Pub,Coffee Shop,Yoga Studio,Ethiopian Restaurant,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [73]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,LONDON,2.0,Coffee Shop,Italian Restaurant,Hotel,Gym / Fitness Center,Pub,Fast Food Restaurant,Sandwich Place,Wine Bar,Salad Place,Restaurant
7,LONDON,2.0,Coffee Shop,Hotel,Pub,Sandwich Place,Salad Place,Restaurant,Gym / Fitness Center,Indian Restaurant,Portuguese Restaurant,Café
9,LONDON,2.0,Hotel,Café,Fish & Chips Shop,Gym / Fitness Center,Supermarket,Portuguese Restaurant,Pub,Bistro,Yoga Studio,Ethiopian Restaurant
10,LONDON,2.0,Coffee Shop,Café,Pub,Italian Restaurant,Bagel Shop,Yoga Studio,Grocery Store,Deli / Bodega,Sushi Restaurant,Hotel
12,LONDON,2.0,Coffee Shop,Café,Pub,Italian Restaurant,Bagel Shop,Yoga Studio,Grocery Store,Deli / Bodega,Sushi Restaurant,Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...
521,LONDON,2.0,Health & Beauty Service,Grocery Store,Liquor Store,Park,Italian Restaurant,Metro Station,Restaurant,Pub,Coffee Shop,Bookstore
522,"LONDON, WOODFORD GREEN",2.0,Hotel,Pub,Plaza,Restaurant,Grocery Store,Bookstore,Monument / Landmark,Spa,Pizza Place,Italian Restaurant
525,LONDON,2.0,Coffee Shop,Grocery Store,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Bus Stop,Hotel,Asian Restaurant,Supermarket
526,LONDON,2.0,Grocery Store,Indian Restaurant,Chinese Restaurant,Bus Stop,Fried Chicken Joint,English Restaurant,Turkish Restaurant,Mediterranean Restaurant,Bar,Wine Shop


In [74]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
167,"LONDON, WELLING",3.0,Bus Stop,Yoga Studio,Food Court,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
259,LONDON,3.0,Bus Stop,Gastropub,Pub,Grocery Store,Yoga Studio,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
457,"LONDON, ERITH",3.0,Bus Stop,Yoga Studio,Food Court,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [75]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,4.0,Bed & Breakfast,Hotel,Pub,Clothing Store,Ethiopian Restaurant,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
172,LONDON,4.0,Hotel,Bed & Breakfast,Food & Drink Shop,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [76]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5, london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

,town,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
377,"HARROW, STANMOREEDGWARE, LONDON",5.0,Warehouse Store,Yoga Studio,English Restaurant,Flower Shop,Fishing Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


# VISUALIZE MAP OF PARIS

In [77]:
# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

In [78]:
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], paris_combined_data['Latitude'], paris_combined_data['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [79]:

venues_in_Paris.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,Bakery
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,Plaza
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,RAP,Gourmet Shop
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,Wine Bar
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,Vegetarian / Vegan Restaurant


In [80]:
venues_in_Paris.shape

(333, 5)

In [81]:
venues_in_Paris.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Argentinian Restaurant,PARIS-6E-ARRONDISSEMENT,48.848968,2.332671,Clasico Argentino
Art Gallery,PARIS-6E-ARRONDISSEMENT,48.863054,2.359361,Institut Hongrois - Collegium Hungaricum
Art Museum,PARIS-6E-ARRONDISSEMENT,48.862630,2.357362,Musée du Luxembourg
Arts & Crafts Store,PARIS-4E-ARRONDISSEMENT,48.854228,2.357362,Melodies graphiques
Asian Restaurant,PARIS-17E-ARRONDISSEMENT,48.887337,2.307486,Guocheng
...,...,...,...,...
Venezuelan Restaurant,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Aji Dulce
Vietnamese Restaurant,PARIS-9E-ARRONDISSEMENT,48.876896,2.337460,Moy Goi Cuon Bar
Wine Bar,PARIS-9E-ARRONDISSEMENT,48.876896,2.359361,Vingt Vins d'Art


In [82]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Dealership,Bakery,Bar,Bed & Breakfast,...,Theater,Theme Park,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:

Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

,Neighbourhood,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Dealership,Bakery,Bar,...,Theater,Theme Park,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [84]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

,Neighbourhood,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Dealership,Bakery,Bar,...,Theater,Theme Park,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,PARIS-10E-ARRONDISSEMENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632,0.052632,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
1,PARIS-11E-ARRONDISSEMENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.090909,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,PARIS-12E-ARRONDISSEMENT,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,PARIS-13E-ARRONDISSEMENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,PARIS-14E-ARRONDISSEMENT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,Coffee Shop,French Restaurant,Beer Garden,Sandwich Place,Shopping Mall,Garden,Mediterranean Restaurant,Cosmetics Shop,Lounge,Bistro
1,PARIS-11E-ARRONDISSEMENT,Grocery Store,Bar,Performing Arts Venue,Pastry Shop,Nightclub,Sandwich Place,Cocktail Bar,Café,Bed & Breakfast,Ethiopian Restaurant
2,PARIS-12E-ARRONDISSEMENT,Auto Dealership,Women's Store,Gym / Fitness Center,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant
3,PARIS-13E-ARRONDISSEMENT,Chinese Restaurant,Women's Store,Gym,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Bistro,Plaza,Pizza Place,Fast Food Restaurant,Food & Drink Shop,Multiplex,Sandwich Place,Mobile Phone Shop,Café


# MODEL BUILDING

In [86]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(n_clusters=5, random_state=0)

In [87]:
kmeans_Paris.labels_

array([4, 4, 0, 2, 4, 1, 3, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4],
      dtype=int32)

In [88]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [89]:
paris_data = paris_combined_data

paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')

paris_data.head()

,postal_code,nom_comm,nom_dept,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,5,French Restaurant,Bistro,Bakery,Plaza,Restaurant,Gourmet Shop,Café,Coffee Shop,Gym / Fitness Center,Latin American Restaurant
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,5,French Restaurant,Bistro,Salad Place,Gym,Plaza,Pizza Place,Perfume Shop,Nightclub,Music Store,Israeli Restaurant
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,5,Grocery Store,Bar,Performing Arts Venue,Pastry Shop,Nightclub,Sandwich Place,Cocktail Bar,Café,Bed & Breakfast,Ethiopian Restaurant
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,5,Hotel,Café,Restaurant,Moroccan Restaurant,Sandwich Place,French Restaurant,Boutique,Perfume Shop,Dessert Shop,Pizza Place
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,5,Hotel,French Restaurant,Tea Room,Sandwich Place,Café,Convenience Store,Boutique,Bookstore,Argentinian Restaurant,Bakery


In [90]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

In [91]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

# Examining Clusters

In [92]:

paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,PARIS-12E-ARRONDISSEMENT,1,Auto Dealership,Women's Store,Gym / Fitness Center,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant


In [93]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,PARIS-20E-ARRONDISSEMENT,2,Health Food Store,Bakery,Bookstore,Gym / Fitness Center,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop
18,PARIS-15E-ARRONDISSEMENT,2,Theme Park,Indian Restaurant,Bakery,Gym,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Grocery Store,Discount Store


In [94]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,PARIS-13E-ARRONDISSEMENT,3,Chinese Restaurant,Women's Store,Gym,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,French Restaurant


In [95]:

paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,PARIS-16E-ARRONDISSEMENT,4,Park,Women's Store,Grocery Store,Discount Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop


In [96]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

,nom_comm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,5,French Restaurant,Bistro,Bakery,Plaza,Restaurant,Gourmet Shop,Café,Coffee Shop,Gym / Fitness Center,Latin American Restaurant
1,PARIS-2E-ARRONDISSEMENT,5,French Restaurant,Bistro,Salad Place,Gym,Plaza,Pizza Place,Perfume Shop,Nightclub,Music Store,Israeli Restaurant
2,PARIS-11E-ARRONDISSEMENT,5,Grocery Store,Bar,Performing Arts Venue,Pastry Shop,Nightclub,Sandwich Place,Cocktail Bar,Café,Bed & Breakfast,Ethiopian Restaurant
3,PARIS-3E-ARRONDISSEMENT,5,Hotel,Café,Restaurant,Moroccan Restaurant,Sandwich Place,French Restaurant,Boutique,Perfume Shop,Dessert Shop,Pizza Place
4,PARIS-6E-ARRONDISSEMENT,5,Hotel,French Restaurant,Tea Room,Sandwich Place,Café,Convenience Store,Boutique,Bookstore,Argentinian Restaurant,Bakery
5,PARIS-4E-ARRONDISSEMENT,5,French Restaurant,Wine Bar,Italian Restaurant,Hostel,Coffee Shop,Burgundian Restaurant,Restaurant,Gourmet Shop,Scandinavian Restaurant,Memorial Site
6,PARIS-1ER-ARRONDISSEMENT,5,Café,Plaza,Coffee Shop,French Restaurant,Hotel,Smoke Shop,Sculpture Garden,Shopping Mall,Ramen Restaurant,Bistro
7,PARIS-8E-ARRONDISSEMENT,5,French Restaurant,Hotel,Italian Restaurant,Café,Bar,Hotel Bar,Bistro,Middle Eastern Restaurant,Cocktail Bar,Resort
10,PARIS-5E-ARRONDISSEMENT,5,French Restaurant,Bar,Creperie,Ice Cream Shop,Lebanese Restaurant,Sushi Restaurant,Plaza,Restaurant,Pub,Pool
11,PARIS-19E-ARRONDISSEMENT,5,French Restaurant,Burger Joint,Supermarket,Chinese Restaurant,Bar,Spa,Fast Food Restaurant,Food & Drink Shop,Farmers Market,Falafel Restaurant


# Results and Discussion

The neighbourhoods of London are very mulitcultural. There are a lot of different cusines including Indian, Italian, Turkish and Chinese. London seems to take a step further in this direction by having a lot of Restaurants, bars, juice bars, coffee shops, Fish and Chips shop and Breakfast spots. It has a lot of shopping options too with that of the Flea markets, flower shops, fish markets, Fishing stores, clothing stores. The main modes of transport seem to be Buses and trains. For leisure, the neighbourhoods are set up to have lots of parks, golf courses, zoo, gyms and Historic sites.

Overall, the city of London offers a multicultural, diverse and certainly an entertaining experience.

Paris is relatively small in size geographically. It has a wide variety of cusines and eateries including French, Thai, Cambodian, Asian, Chinese etc. There are a lot of hangout spots including many Restaurants and Bars. Paris has a lot of Bistro's. Different means of public transport in Paris which includes buses, bikes, boats or ferries. For leisure and sight seeing, there are a lot of Plazas, Trails, Parks, Historic sites, clothing shops, Art galleries and Museums. Overall, Paris seems like the relaxing vacation spot with a mix of lakes, historic spots and a wide variety of cusines to try out.